In [1]:
import os
import re
from os.path import exists
import pandas as pd
import csv
from datetime import date

In [2]:
html_location = '/home/justdial/Documents/keywords_rank_input_ws/'

In [3]:
rows = []
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
def cleanhtml(raw_html):
    cleantext =re.sub(CLEANR, '', str(raw_html))
    return cleantext

In [4]:
out_csv = 'insta_urls_ws_new2.csv'

csv_exists = exists(out_csv)
if csv_exists:
    print(f"csv exists in path {out_csv},removing now ....")
    os.remove(out_csv)
    
#df = pd.DataFrame(columns=['main_category','leaf_category','ns_price','ns_series','ns_parent_brand','ns_brand','ns_product_UID','ns_model_number','ns_product_group','ns_website_prod.name','ns_displayname','ns_description','ns_features','ns_specifications','ns_extraspec','ns_image_urls','ns_url','ns_active_flag','ns_launch_date','country_of_origin','manufacturer','manufactured_details','importers_detail','importer','product_warranty','ns_video_url','pdf_url','aplus_contents'])
df = pd.DataFrame(columns=['search_string','city','instagram_url','unique','description','phone1','phone2','phone3','date'])
header_list=df.columns
print(f"creating new CSV...")
df.to_csv(out_csv,sep="|",index=False,header=False)

csv exists in path insta_urls_ws_new2.csv,removing now ....
creating new CSV...


In [5]:
count = 0
count2 = 0
ic = 0

for i in os.listdir(html_location):
    if i.endswith(".html"):
        count += 1
        file_path = os.path.join(html_location,i)
#         print(file_path)
        HTMLFile = open(file_path, "r")
        page = HTMLFile.read()
        page = re.sub('\\n','',str(page))
        page = re.sub('<!--(.*?)-->','',str(page))
        page = re.sub('#038;','',str(page))
#         print(page)    
        my_dict = {}
#         search_string = ''
        if 'instagram' in page:
            print('file_path ==',file_path)
            count2 += 1
#             print(count2)
            # search_string
            try:
                search_string = re.findall('<title>(.*?)</title>',str(page))[0]
            except:
                search_string = ''
            search_string = search_string.replace('www.instagram.com','www.instagram.com ').replace(' - Google Search','')
            print('search_string --',search_string)

            # city
            try:
                city = re.findall(r',\s*([\w\s]+)$',str(search_string))[0]
            except:
                city = ''
            
#             print('city ==',city)

            today = date.today()
            

            
            # instagram_url
            instagram_url = ''
            insta_urls = []
            ic = 0
            
            if '<div class="A6K0A"' in page:
                print('insta cut found')
#                 ic += 1
                cut_list = page.split('<div class="A6K0A"')
                print('cut_list ==>',len(cut_list))
                print()
                print()
#                 print(cut_list)
                cut_list.pop(0)
                print('cut_list ==>',len(cut_list))
                
                for cut in cut_list:
#                     description_l = re.findall('data-snf="nke7rc"(.*?)<\/div><\/div><\/div><\/div><\/div>',str(cut))
                    
                    insta_urls = re.findall('<a jsname="UWckNb" class="zReHs" href="(.*?)"',str(cut))
                    print('insta_urls -->',len(insta_urls))
                    print('insta_urls',insta_urls)
                    
                    if len(insta_urls):
                        if 'instagram' in insta_urls[0]:
                            print('instagram.com is there')
                            my_dict['instagram_url'] = insta_urls[0]
#                         else:
#                             my_dict['instagram_url'] = ''

                            unique = re.findall(r'(/[^/]+/?)$',str(insta_urls[0]))[0]
                            print('unique -- ',unique)
                            my_dict['unique'] = unique
                
                            my_dict['city'] = city.strip()
                            my_dict['date'] = today.strftime("%d-%m-%Y")
                            my_dict['search_string'] = search_string
                        
                       
                        
                            description_l = re.findall('data-snf="nke7rc"(.*?)<\/div><\/div><\/div><\/div><\/div>',str(cut))



                            if len(description_l):
                                description = cleanhtml(description_l[0])
                                description = description.replace('data-sncf="1">','').replace('data-sncf="1,2">','').replace('❤️','').replace('꓄','').replace('➡️','').replace('🕹️','').replace('✨','').replace('☎️','').replace('•','')
        #                         print(description)
                                my_dict['description'] = description.strip()

        #                         insta_urls = re.findall('<a jsname="UWckNb" class="zReHs" href="(.*?)"',str(description))
        #                         if len(insta_urls):
        #                             my_dict['instagram_url'] = insta_urls[0]
        #                         else:
        #                             my_dict['instagram_url'] = ''


                                phones = re.findall(r'\b(?:\d\s*){7,}\b',str(description))
                                print(len(phones))
                                print(phones)

                                if len(phones) == 0:
                                    my_dict['phone1'] = ''
                                    my_dict['phone2'] = ''
                                    my_dict['phone3'] = ''

                                if len(phones) == 1:
                                    my_dict['phone1'] = phones[0]
                                    my_dict['phone2'] = ''
                                    my_dict['phone3'] = ''
                                if len(phones) == 2:
                                    my_dict['phone1'] = phones[0]
                                    my_dict['phone2'] = phones[1]
                                    my_dict['phone3'] = ''
                                if len(phones) == 3:
                                    my_dict['phone1'] = phones[0]
                                    my_dict['phone2'] = phones[1]
                                    my_dict['phone3'] = phones[2]

                            else:
                                my_dict['description'] = ""


    #                     else:
    #                         my_dict['instagram_url'] = ''
        #                         my_dict['description'] = description

                            df = pd.DataFrame(my_dict, index=[0])
                            list1=list(my_dict.values())
                            print(f"writing to csv .... ")
                            #count+=1
                            with open(out_csv, 'a', newline='') as file:
                                writer = csv.DictWriter(file, fieldnames=header_list,)
                                if file.tell() == 0:
                                    writer.writeheader()# Write the data row
                                writer.writerow(my_dict)
                            print('done')





            
            
#             links = re.findall('<a.*?>.*?</a>',str(page))
# #             print(len(links))
#             for link in links:
#                 if 'href="https://www.instagram.com' in link:
#                     insta_links = re.findall('<a.*?href\=\"https\:\/\/www\.instagram\.com(.*?)".*?>',str(link))
# #                     print('insta links --> ',len(insta_links))
                    
# #                     insta_urls = list(set(insta_links))
                    
# #                     for il in insta_links:
# #                         print('il ==>',il)
# #                         if il not in insta_urls:
# #                             il_url = 'https://www.instagram.com' + il
# #                             insta_urls.append(il_url)
                            
                            
#                     for il in insta_links:
#                         full = 'https://www.instagram.com' + il
# #                         print('full url:', full)
#                         if full not in insta_urls:
#                             insta_urls.append(full)

# #             print('insta_urls --',len(insta_urls))
            
#             if len(insta_urls):
#                 for i in insta_urls:
#                     my_dict['instagram_url'] = i
#             else:
#                 my_dict['instagram_url'] = 'url not found'
                
#     else:
#         my_dict['instagram_url'] = 'url not found'
        
        
            

        
#         print(my_func(audio_path))
# print(count2)
#     my_dict['search_string'] = search_string
    
#     df = pd.DataFrame(my_dict, index=[0])
#     list1=list(my_dict.values())
#     print(f"writing to csv .... ")
#     #count+=1
#     with open(out_csv, 'a', newline='') as file:
#         writer = csv.DictWriter(file, fieldnames=header_list,)
#         if file.tell() == 0:
#             writer.writeheader()# Write the data row
#         writer.writerow(my_dict)
#     print('done')



file_path == /home/justdial/Documents/keywords_rank_input_ws/Naya_Mohalla,_110006,_Delhi_page_4.html
search_string -- www.instagram.com Naya Mohalla, 110006, Delhi
file_path == /home/justdial/Documents/keywords_rank_input_ws/Motia_Bagh_Railway_Colony,_110055,_Delhi_page_6.html
search_string -- www.instagram.com Motia Bagh Railway Colony, 110055, Delhi
file_path == /home/justdial/Documents/keywords_rank_input_ws/Model_Colony,_411016,_Pune_page_7.html
search_string -- www.instagram.com Model Colony, 411016, Pune
file_path == /home/justdial/Documents/keywords_rank_input_ws/NH_4_Faridabad,_121001,_Faridabad_page_8.html
search_string -- www.instagram.com NH 4 Faridabad, 121001, Faridabad
file_path == /home/justdial/Documents/keywords_rank_input_ws/Mv_Extenstion,_562114,_Bangalore_page_6.html
search_string -- www.instagram.com Mv Extenstion, 562114, Bangalore
file_path == /home/justdial/Documents/keywords_rank_input_ws/Mhatardi_Gaon,_400612,_Thane_page_9.html
search_string -- www.instagram.c

file_path == /home/justdial/Documents/keywords_rank_input_ws/Nirnay_Nagar_Sector_1,_382481,_Ahmedabad_page_6.html
search_string -- www.instagram.com Nirnay Nagar Sector 1, 382481, Ahmedabad
file_path == /home/justdial/Documents/keywords_rank_input_ws/May_Field_Garden,_122018,_Gurgaon_page_7.html
search_string -- www.instagram.com May Field Garden, 122018, Gurgaon
file_path == /home/justdial/Documents/keywords_rank_input_ws/Mudiahat,_700128,_Kolkata_page_5.html
search_string -- www.instagram.com Mudiahat, 700128, Kolkata
file_path == /home/justdial/Documents/keywords_rank_input_ws/Moonram_Kattalai,_600069,_Chennai_page_9.html
search_string -- www.instagram.com Moonram Kattalai, 600069, Chennai
file_path == /home/justdial/Documents/keywords_rank_input_ws/Nehru_Vihar,_110054,_Delhi_page_2.html
search_string -- www.instagram.com Nehru Vihar, 110054, Delhi
file_path == /home/justdial/Documents/keywords_rank_input_ws/Murthaza_Nagar-Dabeerpura,_500023,_Hyderabad_page_6.html
search_string -- w

file_path == /home/justdial/Documents/keywords_rank_input_ws/Mirani_Nagar,_400080,_Mumbai_page_8.html
search_string -- www.instagram.com Mirani Nagar, 400080, Mumbai
file_path == /home/justdial/Documents/keywords_rank_input_ws/Mombapur,_501142,_Rangareddy_page_4.html
search_string -- www.instagram.com Mombapur, 501142, Rangareddy
file_path == /home/justdial/Documents/keywords_rank_input_ws/Mathikere,_560054,_Bangalore_page_10.html
search_string -- www.instagram.com Mathikere, 560054, Bangalore
file_path == /home/justdial/Documents/keywords_rank_input_ws/Nilgiri_Chowk-Ambegaon_Budruk,_411046,_Pune_page_3.html
search_string -- www.instagram.com Nilgiri Chowk-Ambegaon Budruk, 411046, Pune
file_path == /home/justdial/Documents/keywords_rank_input_ws/Mehrauli_Ghaziabad,_201001,_Ghaziabad_page_3.html
search_string -- www.instagram.com Mehrauli Ghaziabad, 201001, Ghaziabad
file_path == /home/justdial/Documents/keywords_rank_input_ws/New_Prabhadevi,_400025,_Mumbai_page_4.html
search_string -- 

file_path == /home/justdial/Documents/keywords_rank_input_ws/Mavena_Nagar,_600019,_Chennai_page_9.html
search_string -- www.instagram.com Mavena Nagar, 600019, Chennai
file_path == /home/justdial/Documents/keywords_rank_input_ws/Manickpur,_401202,_Palghar_page_2.html
search_string -- www.instagram.com Manickpur, 401202, Palghar
file_path == /home/justdial/Documents/keywords_rank_input_ws/Mandai,_421302,_Bhiwandi_page_2.html
search_string -- www.instagram.com Mandai, 421302, Bhiwandi
file_path == /home/justdial/Documents/keywords_rank_input_ws/Motilal_Nagar_No_1-Goregaon_West,_400104,_Mumbai_page_7.html
search_string -- www.instagram.com Motilal Nagar No 1-Goregaon West, 400104, Mumbai
file_path == /home/justdial/Documents/keywords_rank_input_ws/Mangammanapalya-Bommanahalli,_560068,_Bangalore_page_10.html
search_string -- www.instagram.com Mangammanapalya-Bommanahalli, 560068, Bangalore
file_path == /home/justdial/Documents/keywords_rank_input_ws/Manthangowd_Gowrelli,_501509,_Rangareddy

file_path == /home/justdial/Documents/keywords_rank_input_ws/Moheman_Sarai,_201001,_Ghaziabad_page_1.html
search_string -- www.instagram.com Moheman Sarai, 201001, Ghaziabad
file_path == /home/justdial/Documents/keywords_rank_input_ws/Muthekhar,_410206,_Navi_Mumbai_page_5.html
search_string -- www.instagram.com Muthekhar, 410206, Navi Mumbai
file_path == /home/justdial/Documents/keywords_rank_input_ws/Nerul_Sector_24,_400706,_Navi_Mumbai_page_4.html
search_string -- www.instagram.com Nerul Sector 24, 400706, Navi Mumbai
file_path == /home/justdial/Documents/keywords_rank_input_ws/Mumbra,_400612,_Thane_page_5.html
search_string -- www.instagram.com Mumbra, 400612, Thane
insta cut found
cut_list ==> 11


cut_list ==> 10
insta_urls --> 1
insta_urls ['https://www.instagram.com/reel/DCjPM5HtCT0/?hl=en']
instagram.com is there
unique --  /?hl=en
writing to csv .... 
done
insta_urls --> 1
insta_urls ['https://www.instagram.com/sp_honda_mumbra/']
instagram.com is there
unique --  /sp_honda_mum

file_path == /home/justdial/Documents/keywords_rank_input_ws/New_Maneklal_Estate-Ghatkopar_West,_400086,_Mumbai_page_6.html
search_string -- www.instagram.com New Maneklal Estate-Ghatkopar West, 400086, Mumbai
file_path == /home/justdial/Documents/keywords_rank_input_ws/Mullai_Nagar-Ennore,_600057,_Chennai_page_6.html
search_string -- www.instagram.com Mullai Nagar-Ennore, 600057, Chennai
file_path == /home/justdial/Documents/keywords_rank_input_ws/Nagegowdanapalya,_560109,_Bangalore_page_2.html
search_string -- www.instagram.com Nagegowdanapalya, 560109, Bangalore
file_path == /home/justdial/Documents/keywords_rank_input_ws/Maxworth_Nagar,_600117,_Chennai_page_7.html
search_string -- www.instagram.com Maxworth Nagar, 600117, Chennai
file_path == /home/justdial/Documents/keywords_rank_input_ws/Moghul_Colony-Mailardevpalli,_500077,_Hyderabad_page_10.html
search_string -- www.instagram.com Moghul Colony-Mailardevpalli, 500077, Hyderabad
file_path == /home/justdial/Documents/keywords_rank

file_path == /home/justdial/Documents/keywords_rank_input_ws/Mundhela_Kalan,_110073,_Delhi_page_10.html
search_string -- www.instagram.com Mundhela Kalan, 110073, Delhi


KeyboardInterrupt: 

In [6]:
print(date)

NameError: name 'date' is not defined